In [62]:
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql import Row

In [63]:
contest_data_sample = sqlContext.sql("select * from ignite.contest_data_sample").repartition(50).cache()

contest_cr = sqlContext.sql("select * from edso_ignite.contest_cr").repartition(200).cache()

contest_savm = sqlContext.sql("select * from edso_ignite.contest_savm").repartition(200).cache()

In [64]:
 #Randomly picking an entry
test = contest_data_sample.sample(False,0.002).select('end_customer_party_ssot_party_id_int_sav_party_id','sales_acct_id','decision_date_time').take(1)
test[0]

Row(end_customer_party_ssot_party_id_int_sav_party_id=26724, sales_acct_id=203715515.0, decision_date_time=u'2014-09-10 13:25:57.0')

In [65]:
test_case = contest_cr.where(F.col('party_id')== test[0][0]).take(1)
test_case[0]

Row(party_id=26724.0, parent_party_id=34983.0, party_name=u'SPARTANBURG REGIONAL HEALTH SERVICES DISTRICT INC', node_type=u'HQ', address1=u'101 E WOOD ST', address2=None, address3=None, address4=None, city=u'SPARTANBURG', county=u'SPARTANBURG', state=u'SC', province=None, postal_code=u'29303', postal_code_extn=u'3040', country_code=u'US', street_name=u'WOOD', street_number=u'101', street_direction=u'E', street_type=u'ST', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2002-11-11 22:59:10.0', end_date=u'4712-12-31 00:00:00.0', program_id=None, request_id=None, created_by=1116.0, last_updated_by=332355.0, creation_date=u'2002-11-11 22:59:10.0', last_update_date=u'2014-11-16 13:12:48.0', certified_date=u'2014-03-07 14:44:48.0')

In [66]:
all_acct_ids = contest_savm.where\
((F.col('postal_code').rlike("^"+test_case[0][12]+"$"))  & (F.col('start_date')>=test[0][2]))#.take(50)
actual_acct_id = all_acct_ids.where(F.col('sales_acct_id')==test[0][1])
print actual_acct_id.count(),'/',all_acct_ids.select('sales_acct_id').count()
print 'Distinct sales_acct_ids: ',all_acct_ids.select('sales_acct_id').distinct().count()

36 / 166
Distinct sales_acct_ids:  65


In [67]:
print 'See correlation with actual: ',test_case[0][2]
actual_acct_id.select('party_name').distinct().show(10,False)

See correlation with actual:  SPARTANBURG REGIONAL HEALTH SERVICES DISTRICT INC
+--------------------------------------+
|party_name                            |
+--------------------------------------+
|SPARTANBURG REGIONAL HEALTHCARE SYSTEM|
|SPARTANBURG REGIONAL MEDICAL CENTER   |
|ADAMS SPARTANBURG                     |
|SRHS                                  |
|SPARTANBURG REGIONAL HEALTH SERVICE   |
|SPARTANBURG COUNTY GOVERNMENT         |
+--------------------------------------+



In [68]:
print 'See correlation with actual: ',test_case[0][4]
actual_acct_id.select('address1').distinct().show(10,False)

See correlation with actual:  101 E WOOD ST
+---------------+
|address1       |
+---------------+
|700 N PINE ST  |
|475 SIMUEL RD  |
|311 N CHURCH ST|
|1001 NORTH PINE|
|101 E WOOD ST  |
+---------------+



In [69]:
print 'See correlation with actual: ',test_case[0][15]
actual_acct_id.select('street_name').distinct().show(10,False)

See correlation with actual:  WOOD
+-----------+
|street_name|
+-----------+
|PINE       |
|PO BOX     |
|NORTH      |
|WOOD       |
|null       |
|CHURCH     |
+-----------+

